In [1]:
#evaluate the model 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import joblib


In [4]:
df = pd.read_csv("../reports/matches_features.csv") 
print("Missing targets before cleanup:", df["target"].isna().sum()) 
print(df[df["target"].isna()][["date", "home_team", "away_team", "score"]]) 
df = df.dropna(subset=["target"]) # Extract features and target
features = [ "home_team_avg_goals_for", "home_team_avg_goals_against", "away_team_avg_goals_for", "away_team_avg_goals_against", "year", "month", "weekday" ] 
X = df[features] 
y = df["target"] 
# Split again (ensure consistency with Notebook 03) 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y ) # Load trained model 
model = joblib.load("../models/random_forest_model.pkl")


Missing targets before cleanup: 0
Empty DataFrame
Columns: [date, home_team, away_team, score]
Index: []


In [5]:
# Make predictions
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {acc:.3f}")

# Classification report
print("\n Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Home Win", "Draw", "Away Win"],
            yticklabels=["Home Win", "Draw", "Away Win"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# visualization 
if hasattr(model, "feature_importances_"):
    importances = pd.Series(model.feature_importances_, index=features)
    importances.sort_values(ascending=True).plot(kind="barh", figsize=(8,5))
    plt.title("Feature Importance in Random Forest Model")
    plt.xlabel("Importance Score")
    plt.show()


results_df = X_test.copy()
results_df["true_target"] = y_test
results_df["predicted_target"] = y_pred
results_df.to_csv("model_predictions.csv", index=False)



ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- avg_goals_against_combined
- avg_goals_for_combined
